# Multiple Dispatch
## The secret sauce behind Julia's performance

In this notebook, we will demonstrate one of the core pieces of Julia's design. This property, called *Multiple Dispatch* is the fact that at runtime, the Julia compiler will specialize function calls for each specific combination of input types that is provided.

Another benefit of Multiple Dispatch is that it allows for amazing flexibility when writing functions. This has allowed the Julia community to have an impressive amount of code reuse, as it's not uncommon to see one user's functions work on another user's types *out of the box*.

## A very simple example

Let's us start with a toy example to illustrate how Multiple Dispatch plays with Julia's type system. Consider the following function.

In [11]:
f(x) = 2*x^2 + x

f (generic function with 1 method)

This is a generic function, meaning it will take any type and gladly try to compute it. In this case, this is meant to take in numeric types as arguments, and it will specialize every time we give it a new input type.

In [12]:
f(1)

3

In [13]:
f(1.0)

3.0

As we can see above, `f` returns an `Int` when provided with and `Int`, and a `Float64`, when we provide it with that type.

We can actually look under the hood using the macro `@code_llvm` to see what happens during a given call:

In [14]:
@code_llvm f(1.0)

;  @ In[11]:1 within `f`
define double @julia_f_1594(double %0) #0 {
top:
; ┌ @ intfuncs.jl:312 within `literal_pow`
; │┌ @ float.jl:405 within `*`
    %1 = fmul double %0, %0
; └└
; ┌ @ promotion.jl:380 within `*` @ float.jl:405
   %2 = fmul double %1, 2.000000e+00
; └
; ┌ @ float.jl:399 within `+`
   %3 = fadd double %2, %0
; └
  ret double %3
}


However, let us now consider the following function:

In [6]:
g(x) = 2*x^2 + x/2

g (generic function with 2 methods)

When we call `g` on an integer, something wierd happens

In [7]:
g(1.0)

2.5

In [8]:
g(1)


2

What happened? Well the division operator `/` in Julia computes the *exact division*. For integer division we need to use the `div` function (or equivalently the `÷` (`\div<TAB>`)). Let us correct our problem by defining a new method for `g`.

In [1]:
g(n::Int) = 2*n^2 + div(n,2)

g (generic function with 1 method)

Now if we evaluate `g` on an `Int`, it returns an `Int` as we wanted

In [9]:
g(1)

2

Problem solved? Not quite, our new method only applies to `Int` which is actually `Int64` (64 bit signed integers). But there are other integer types:

In [10]:
g(Int32(1))

2.5

To fix this, we could define a new method for `Int32`, but we would also have to do that for every other integer type. And then if someone else uses our code and brings in some new integer types they'll have to define new methods for those as well! How tedious!

Luckily, we don't actually need to do that. We just need to define a single method for `Integer`, which is the abstract type for all integers.

In [11]:
g(n::Integer) = 2*n^2 + div(n,2)

g (generic function with 3 methods)

In [12]:
g(Int32(1))

2

## A more advanced example

In this next example, we'll showcase some of the more interesting things that we can do with multiple dispatch. We'll implement a basic version of *Forward mode Automatic Differentiation* in about ten lines of code.

This example is largely adapted from https://www.youtube.com/watch?v=vAp6nUMrKYg

Let's start by defining a new type called `Dual`. It represents pairs of numbers of the form 
$$ x + \epsilon y $$
where $\epsilon$ is a bit like the imaginary unit $i$, except it satisfies $\epsilon^2 = 0$.

Another way to interpret them is that $x$ represents the value of some function evaluated at some point, while $y$ represents the value of the *derivative* of that function at the same point.

In [1]:
struct Dual <: Number
    x::Float64
    y::Float64
end

Since `Dual` is a subtype of `Number`, we'll need to define arithmetic operations for it, so that it can play with other Number types.

In [2]:
import Base: +, -, *, /, convert, promote_rule, show

# The following four lines define arithmetic operations on Duals. Notice how the rules for * and / are the same as derivative rules for multiplication and division
+(a::Dual, b::Dual) = Dual(a.x + b.x, a.y + b.y)
-(a::Dual, b::Dual) = Dual(a.x - b.x, a.y - b.y)
*(a::Dual, b::Dual) = Dual(a.x*b.x, a.y*b.x + a.x*b.y)
/(a::Dual, b::Dual) = Dual(a.x/b.x, (a.y*b.x - a.x*b.y)/(b.x^2))

# These three lines tell how to convert between real numbers and Dual, and how to pretty print Dual numbers.
convert(::Type{Dual}, x::Real) = Dual(x, zero(x))
promote_rule(::Type{Dual}, ::Type{<:Number}) = Dual
show(io::IO, d::Dual) = print(io, d.x, " + ", d.y," ϵ")

show (generic function with 304 methods)

We can try out our new type on a simple function to check that it works.

In [15]:
f(1), f(1.0), f(Dual(1,1))

(3, 3.0, 3.0 + 5.0 ϵ)

So far, so good. Let's try a more complicated function.

In [7]:
"""
Compute the square root of x using the Babylonian algorithm
"""
function babylonian(x; nmax = 10)
    t = (1+x)/2
    for i in 2:nmax
        t = (t + x/t)/2
    end
    return t
end

babylonian

In [8]:
babylonian(π), √π, babylonian(2), √2

(1.7724538509055159, 1.7724538509055159, 1.414213562373095, 1.4142135623730951)

In [9]:
x = Dual(2, 1)
babylonian(x)

1.414213562373095 + 0.35355339059327373 ϵ

It just worked!

In [10]:
0.5/√2

0.35355339059327373

Let that sink in. In just about ten lines of code, we just defined a barebones implementation of automatic differentiation that will work with almost all numerical Julia functions (provided we define a couple more primitives).

This is impossible to do in pure Python while keeping it fast, and to do that in C/C++/Fortran, we'd either have to rewrite the entire math library to support our types, or write a special compiler.